In [9]:
import numpy as np
def leaky_relu(z):
    return np.where(z > 0, z, z * 0.01)

def softmax(z):
    if len(z.shape) > 1:
        # Softmax for matrix
        max_matrix = np.max(z, axis=0)
        stable_z = z - max_matrix
        e = np.exp(stable_z)
        a = e / np.sum(e, axis=0, keepdims=True)
    else:
        # Softmax for vector
        vector_max_value = np.max(z)
        a = (np.exp(z - vector_max_value)) / sum(np.exp(z - vector_max_value))

    assert a.shape == z.shape

    return a

In [ ]:

print('\n\n----- One-hot vector representation of nodes. Shape(n,n)\n')
X = np.eye(5, 5)
n = X.shape[0]
np.random.shuffle(X)
print(X)

print('\n\n----- Embedding dimension\n')
emb = 3
print(emb)

print('\n\n----- Weight Matrix. Shape(emb, n)\n')
W = np.random.uniform(-np.sqrt(1. / emb), np.sqrt(1. / emb), (emb, n))
print(W)

print('\n\n----- Adjacency Matrix (undirected graph). Shape(n,n)\n')
A = np.random.randint(2, size=(n, n))
np.fill_diagonal(A, 1)  
A = (A + A.T)
A[A > 1] = 1
print(A)

In [2]:
print('\n\n----- Linear Transformation. Shape(n, emb)\n')
z1 = X.dot(W.T)
print(z1)



----- Linear Transformation. Shape(n, emb)

[[-0.16565732 -0.01631104  0.08638784]
 [ 0.44534168  0.51071103  0.21891362]
 [-0.37986297 -0.00899712 -0.23351036]
 [-0.06611432 -0.22554181  0.46799425]
 [ 0.17553066 -0.06967223  0.43153729]]


In [3]:
# equation (2) - First part
print('\n\n----- Concat hidden features to represent edges. Shape(len(emb.concat(emb)), number of edges)\n')
edge_coords = np.where(A==1)
h_src_nodes = z1[edge_coords[0]]
h_dst_nodes = z1[edge_coords[1]]
z2 = np.concatenate((h_src_nodes, h_dst_nodes), axis=1)



----- Concat hidden features to represent edges. Shape(len(emb.concat(emb)), number of edges)



In [14]:
h_src_nodes

array([[-0.16565732, -0.01631104,  0.08638784],
       [-0.16565732, -0.01631104,  0.08638784],
       [-0.16565732, -0.01631104,  0.08638784],
       [-0.16565732, -0.01631104,  0.08638784],
       [ 0.44534168,  0.51071103,  0.21891362],
       [ 0.44534168,  0.51071103,  0.21891362],
       [ 0.44534168,  0.51071103,  0.21891362],
       [ 0.44534168,  0.51071103,  0.21891362],
       [-0.37986297, -0.00899712, -0.23351036],
       [-0.37986297, -0.00899712, -0.23351036],
       [-0.37986297, -0.00899712, -0.23351036],
       [-0.37986297, -0.00899712, -0.23351036],
       [-0.06611432, -0.22554181,  0.46799425],
       [-0.06611432, -0.22554181,  0.46799425],
       [-0.06611432, -0.22554181,  0.46799425],
       [-0.06611432, -0.22554181,  0.46799425],
       [ 0.17553066, -0.06967223,  0.43153729],
       [ 0.17553066, -0.06967223,  0.43153729],
       [ 0.17553066, -0.06967223,  0.43153729]])

In [4]:
edge_coords

(array([0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4]),
 array([0, 1, 2, 3, 0, 1, 2, 4, 0, 1, 2, 3, 0, 2, 3, 4, 1, 3, 4]))

In [6]:
h_src_nodes.shape

(19, 3)

In [7]:
h_dst_nodes.shape

(19, 3)

In [10]:
# Concatenation tests
assert len(edge_coords[1]) == z2.shape[0], "The number of edges in A is not equal to the number of concat edges"
test_value = np.array([-0.11941829, -0.12942953, 0.19600584, 0.5029172, 0.3998854, -0.21561317])
assert z2[4 ,:].tolist().sort()  == test_value.tolist().sort(), "Something went wrong in the concat process"
print(z2)

print('\n\n----- Attention coefficients. Shape(1, len(emb.concat(emb)))\n')
att = np.random.rand(1, z2.shape[1])
print(att)

print('\n\n----- Edge representations combined with the attention coefficients. Shape(1, number of edges)\n')
z2_att = z2.dot(att.T)
print(z2_att)

print('\n\n----- Leaky Relu. Shape(1, number of edges)')
e = leaky_relu(z2_att)
print(e)

[[-0.16565732 -0.01631104  0.08638784 -0.16565732 -0.01631104  0.08638784]
 [-0.16565732 -0.01631104  0.08638784  0.44534168  0.51071103  0.21891362]
 [-0.16565732 -0.01631104  0.08638784 -0.37986297 -0.00899712 -0.23351036]
 [-0.16565732 -0.01631104  0.08638784 -0.06611432 -0.22554181  0.46799425]
 [ 0.44534168  0.51071103  0.21891362 -0.16565732 -0.01631104  0.08638784]
 [ 0.44534168  0.51071103  0.21891362  0.44534168  0.51071103  0.21891362]
 [ 0.44534168  0.51071103  0.21891362 -0.37986297 -0.00899712 -0.23351036]
 [ 0.44534168  0.51071103  0.21891362  0.17553066 -0.06967223  0.43153729]
 [-0.37986297 -0.00899712 -0.23351036 -0.16565732 -0.01631104  0.08638784]
 [-0.37986297 -0.00899712 -0.23351036  0.44534168  0.51071103  0.21891362]
 [-0.37986297 -0.00899712 -0.23351036 -0.37986297 -0.00899712 -0.23351036]
 [-0.37986297 -0.00899712 -0.23351036 -0.06611432 -0.22554181  0.46799425]
 [-0.06611432 -0.22554181  0.46799425 -0.16565732 -0.01631104  0.08638784]
 [-0.06611432 -0.22554181

In [38]:
# a_input = torch.cat([h.repeat(1, N).view(N * N, -1), h.repeat(N, 1)], dim=1).view(N, -1, 2 * out_features)a
N = z1.shape[0]
z2 = np.concatenate((np.repeat(z1,N,axis=0).reshape(N*N,-1) , np.tile(z1, (N,1))),axis=1)
z2_att = z2.dot(att.T)
print(z2_att)
print('\n\n----- Leaky Relu. Shape(1, number of edges)')
e = leaky_relu(z2_att)
print(e)


[[-0.17687181]
 [ 0.51104787]
 [-0.479485  ]
 [-0.00298638]
 [ 0.19710452]
 [ 0.44181808]
 [ 1.12973775]
 [ 0.13920488]
 [ 0.6157035 ]
 [ 0.8157944 ]
 [-0.41880626]
 [ 0.26911342]
 [-0.72141945]
 [-0.24492083]
 [-0.04482994]
 [-0.0515746 ]
 [ 0.63634508]
 [-0.35418779]
 [ 0.12231083]
 [ 0.32240173]
 [ 0.16192844]
 [ 0.84984812]
 [-0.14068475]
 [ 0.33581387]
 [ 0.53590476]]


----- Leaky Relu. Shape(1, number of edges)
[[-1.76871805e-03]
 [ 5.11047871e-01]
 [-4.79484996e-03]
 [-2.98637938e-05]
 [ 1.97104516e-01]
 [ 4.41818076e-01]
 [ 1.12973775e+00]
 [ 1.39204885e-01]
 [ 6.15703502e-01]
 [ 8.15794397e-01]
 [-4.18806260e-03]
 [ 2.69113416e-01]
 [-7.21419451e-03]
 [-2.44920834e-03]
 [-4.48299385e-04]
 [-5.15745956e-04]
 [ 6.36345081e-01]
 [-3.54187787e-03]
 [ 1.22310830e-01]
 [ 3.22401726e-01]
 [ 1.61928440e-01]
 [ 8.49848116e-01]
 [-1.40684752e-03]
 [ 3.35813865e-01]
 [ 5.35904761e-01]]


In [11]:
# equation (3)
print('\n\n----- Edge scores as matrix. Shape(n,n)\n')
e_matr = np.zeros(A.shape)
e_matr[edge_coords[0], edge_coords[1]] = e.reshape(-1,)
print(e_matr)

print('\n\n----- For each node, normalize the edge (or neighbor) contributions using softmax\n')
alpha0 = softmax(e_matr[:,0][e_matr[:,0] != 0]) 
alpha1 = softmax(e_matr[:,1][e_matr[:,1] != 0])
alpha2 = softmax(e_matr[:,2][e_matr[:,2] != 0])
alpha3 = softmax(e_matr[:,3][e_matr[:,3] != 0])
alpha4 = softmax(e_matr[:,4][e_matr[:,4] != 0])
alpha = np.concatenate((alpha0, alpha1, alpha2, alpha3, alpha4))
print(alpha)

print('\n\n----- Normalized edge score matrix. Shape(n,n)\n')
A_scaled = np.zeros(A.shape)
A_scaled[edge_coords[0], edge_coords[1]] = alpha.reshape(-1,)
print(A_scaled)



----- Edge scores as matrix. Shape(n,n)

[[-1.76871805e-03  5.11047871e-01 -4.79484996e-03 -2.98637938e-05
   0.00000000e+00]
 [ 4.41818076e-01  1.12973775e+00  1.39204885e-01  0.00000000e+00
   8.15794397e-01]
 [-4.18806260e-03  2.69113416e-01 -7.21419451e-03 -2.44920834e-03
   0.00000000e+00]
 [-5.15745956e-04  0.00000000e+00 -3.54187787e-03  1.22310830e-01
   3.22401726e-01]
 [ 0.00000000e+00  8.49848116e-01  0.00000000e+00  3.35813865e-01
   5.35904761e-01]]


----- For each node, normalize the edge (or neighbor) contributions using softmax

[0.21943665 0.34194517 0.2189064  0.21971177 0.19822137 0.36799682
 0.15562511 0.27815671 0.24074799 0.27803595 0.24016624 0.24104983
 0.2209045  0.2203707  0.24965281 0.30907199 0.4225795  0.25800654
 0.31941396]


----- Normalized edge score matrix. Shape(n,n)

[[0.21943665 0.34194517 0.2189064  0.21971177 0.        ]
 [0.19822137 0.36799682 0.15562511 0.         0.27815671]
 [0.24074799 0.27803595 0.24016624 0.24104983 0.        ]
 [0.2209

In [12]:

# equation (4)
print('\n\nNeighborhood aggregation (GCN) scaled with attention scores (GAT). Shape(n, emb)\n')
ND_GAT = A_scaled.dot(z1)
print(ND_GAT)



Neighborhood aggregation (GCN) scaled with attention scores (GAT). Shape(n, emb)

[[ 0.01825062  0.11953221  0.14552005]
 [ 0.12075632  0.16392686  0.18137835]
 [-0.02322777  0.08154156  0.13839218]
 [-0.08255913 -0.08342676  0.21783679]
 [ 0.22720128  0.13537046  0.35109302]]


Equation (1) is a linear transformation of the lower layer embedding $h_i^{(l)}$ and $W^{(l)}$ is its learnable weight matrix. This transformation is useful to achieve a sufficient expressive power to transform input features (in our example one-hot vectors) into high-level and dense features.
Equation (2) computes a pair-wise un-normalized attention score between two neighbors. Here, it first concatenates the $z$ embeddings of the two nodes, where $||$ denotes concatenation, then takes a dot product of it and a learnable weight vector $\vec a^{(l)}$, and applies a LeakyReLU in the end. This form of attention is usually called additive attention, contrast with the dot-product attention in the Transformer model. The attention score indicates the importance of a neighbor node in the message passing framework.
Equation (3) applies a softmax to normalize the attention scores on each node’s incoming edges.
Equation (4) is similar to GCN. The embeddings from neighbors are aggregated together, scaled by the attention scores.